In [ ]:
#@title Get a smaller version of the Lakh MIDI Dataset v0.1
%%capture
!wget http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
!tar xvf clean_midi.tar.gz
!rm clean_midi.tar.gz

dataset_path = "/content/clean_midi"

In [ ]:
%%capture
!pip install pretty_midi

In [ ]:
from pathlib import Path

# Paths to the files of the dataset
midi_paths = list(Path("clean_midi").resolve().glob("**/*.mid"))

## Cleaning of files

In [ ]:
import os
import shutil

midis_dir = "midis"
os.makedirs(midis_dir, exist_ok=True)

for i, midi_path in enumerate(midi_paths):
  new_midi_path = os.path.join(midis_dir, f"{i}.mid")
  shutil.move(str(midi_path), new_midi_path)

midis = list(Path("midis").resolve().glob("**/*.mid"))

## MIDI2Tokens

In [ ]:
import copy
import pretty_midi

class midi_to_tokens():
    def __init__(self, path, steps_per_beat=12, limit=None):
        self.steps_per_beat = steps_per_beat
        try:
          self.pm = pretty_midi.PrettyMIDI(path)
        except OSError as e:
          print(f"Error reading MIDI file: {e}")
          return
        self.dbs = self.pm.get_downbeats().tolist() + [self.pm.get_end_time()] # dbs := downbeats
        self.tokens_seqs = self._tokenize(limit=limit)

    def __call__(self):
        return ' '.join(self.tokens)

    def _time_to_step(self, time):
        return round(self.pm.time_to_tick(time) / self.pm.resolution * self.steps_per_beat)

    def _event_to_tokens(self, event):
        if event in ('bar', 'beat'):
            return [event]
        elif isinstance(event, pretty_midi.containers.Note):
            return [f'note_{event.pitch}', f'len_{self._time_to_step(event.end) - self._time_to_step(event.start)}']

    def _trim_note(self, note, start, end):
        n = copy.copy(note)
        n.start, n.end = max(n.start, start), min(n.end, end)
        return n

    def _tokenize(self, start_measure=1, end_measure=None, limit=None):
        start, end = self.dbs[start_measure - 1], self.dbs[end_measure or -1]

        notes = []
        for inst in self.pm.instruments:
            notes += inst.notes
        notes.sort(key=lambda x: (x.start, -x.pitch))

        events = []
        events += [(self._time_to_step(db), 'bar') for db in self.dbs if start <= db < end]
        events += [(self._time_to_step(b), 'beat') for b in set(self.pm.get_beats()) - set(self.dbs) if start <= b < end] # beats without downbeats
        events += [(self._time_to_step(max(n.start, start)), self._trim_note(n, start, end)) for n in notes if start <= n.start < end or start < n.end <= end]
        events.sort(key=lambda x: x[0])

        tokens_sequences = []
        last_beat = 0
        if limit is not None:
          event_chunks = [events[i:i + limit] for i in range(0, len(events), limit)]
        else:
          event_chunks = [events]

        for chunk in event_chunks:
            tokens = []
            try:
                for step, event in chunk:
                    if event in ('bar', 'beat'):
                        last_beat = step
                    if step - last_beat:
                        tokens.append(f'pos_{step - last_beat}')
                    tokens += self._event_to_tokens(event)
                tokens_sequences.append(tokens)
            except Exception as e:
                print(f"Error while translating events to tokens: {e}")

        return tokens_sequences

    def measures(self, start_measure=1, end_measure=None):
        return self._tokenize(start_measure, end_measure)

## Tokens2MIDI

In [ ]:
class TokensToMidi:
    def __init__(self, tokens, steps_per_beat=12, ticks_per_beat=960, tempo=120):
        self.tokens = tokens
        self.steps_per_beat = steps_per_beat
        self.ticks_per_step = ticks_per_beat // steps_per_beat
        self.tempo = tempo
        self.ticks_per_beat = ticks_per_beat

    def _ticks_to_time(self, ticks):
        return ticks * 60 / (self.tempo * self.ticks_per_beat)

    def generate_midi(self, path):
        pm = pretty_midi.PrettyMIDI(initial_tempo=self.tempo)
        instrument = pretty_midi.Instrument(program=38)

        time = 0
        last_beat = 0

        i = 0
        while i < len(self.tokens):
            token = self.tokens[i]

            if token == "bar":
                time += self._ticks_to_time(self.ticks_per_step * self.steps_per_beat)
                last_beat = time
            elif token == "beat":
                time = last_beat
                last_beat = time
            elif token.startswith("pos_"):
                position = int(token.split("_")[1])
                time = last_beat + self._ticks_to_time(self.ticks_per_step * position)
            elif token.startswith("note_"):
                pitch = int(token.split("_")[1])
                try:
                  length_token = self.tokens[i + 1]
                  length = int(length_token.split("_")[1])
                except IndexError:
                  length = 1
                duration = self._ticks_to_time(self.ticks_per_step * length)

                note = pretty_midi.Note(
                    velocity=100,
                    pitch=pitch,
                    start=time,
                    end=time + duration
                )
                instrument.notes.append(note)

                i += 1
            i += 1

        pm.instruments.append(instrument)
        pm.write(path)
        return pm

## Define a utility class "Tokenizer"

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tqdm import tqdm

class Tokenizer():
    def __init__(self, limit=None):
        self._encoder = LabelEncoder()
        self.PAD_id = 0
        self.BOS_id = 1
        self.EOS_id = 2
        self.limit = limit

    def _tokenize(self, midi_paths):
      """
      midi_paths: list of paths to MIDI files
      returns: list of lists of string tokens
      """
      tokens = []
      for path in tqdm(midi_paths):
        try:
          seqs = midi_to_tokens(str(path), steps_per_beat=12, limit=self.limit).tokens_seqs
          for seq in seqs:
            seq.insert(0, "Start")
            seq.append("End")
            tokens.append(seq)
        except AttributeError:
          print(f"Error reading MIDI file: {path}")
          continue
        except Exception as e:
          print(f"There was an unexpected error: {e}")
          continue
      return tokens

    def fit_and_encode(self, midi_paths):
      tokens = self._tokenize(midi_paths)
      flattened_array = np.concatenate([np.array(sublist).flatten() for sublist in tokens])
      self._encoder.fit(flattened_array)
      transformed = [self._encoder.transform(i) for i in tokens]
      self.EOS_id = self._encoder.transform(["End"])[0]
      self.BOS_id = self._encoder.transform(["Start"])[0]
      self.PAD_id = self._encoder.classes_.shape[0]
      self._encoder.classes_ = np.append(self._encoder.classes_, ["Pad"])
      return transformed

    def encode(self, midi_paths):
      tokens = self._tokenize(midi_paths)
      return [self._encoder.transform(i) for i in tokens]

    def decode(self, encoded_tokens, path="reconstructed_midi.mid"):
      string_tokens = [self._encoder.inverse_transform(i) for i in encoded_tokens]
      for i in range(len(string_tokens)):
        midi_reconstructor = TokensToMidi(string_tokens[i])
        midi_reconstructor.generate_midi(str(i) + path)

    def pad(self, encoded_tokens):
      self._seq_length = max(len(arr) for arr in encoded_tokens)
      return np.array([np.pad(arr, (self._seq_length - len(arr), 0), mode='constant', constant_values=self.PAD_id) for arr in encoded_tokens])

    def save(self, path: str):
      np.savetxt(path, self.encoder.classes_, fmt="%s")

    def load(self, path: str):
      self._encoder.classes_ = np.loadtxt(path, dtype="str")

    @property
    def encoder(self):
      return self._encoder

    @property
    def vocab_size(self):
      return self._encoder.classes_.shape[0]

    @property
    def seq_length(self):
      return self._seq_length

    @property
    def pad_id(self):
      return self.PAD_id

    @property
    def bos_id(self):
      return self.BOS_id

    @property
    def eos_id(self):
      return self.EOS_id


## Download trained Tokenizer and tokens

In [ ]:
!gdown 1QrMzoYewqyxv2TJEgwS53-8l6rlpi4RG
!gdown 1dwNkvRopC8gIpDzD2iEFZADe4aeF-w_F

!mv tokens_lim1000_nFiles2500.txt tokens.txt
!mv tokenizer_lim1000_nFiles2500.txt tokenizer.txt

Downloading...
From: https://drive.google.com/uc?id=1trgZB8lrn9APQF3NWQCucAVbC-rLsyd1
To: /content/tokenizer.txt
100% 10.7k/10.7k [00:00<00:00, 31.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=12zSaNlKv2twSeym7LS91pFCxmsKVVaes
From (redirected): https://drive.google.com/uc?id=12zSaNlKv2twSeym7LS91pFCxmsKVVaes&confirm=t&uuid=8d2c6779-3236-495f-8925-75b44252a80c
To: /content/tokens.txt
100% 393M/393M [00:06<00:00, 59.7MB/s]


## Load a previously fitted Tokenizer

In [ ]:
tok = Tokenizer(limit=500)
tok.load("tokenizer.txt")

## Load already tokenized data...

In [ ]:
padded_tokens = np.loadtxt("tokens.txt")
print(f"Loaded tokenized version of {padded_tokens.shape[0]} files")

Loaded tokenized version of 58954 files


## ...or fit the tokenizer

In [ ]:
tok = Tokenizer(limit=500)
encoded_tokens = tok.fit_and_encode(midis[:100])

print(f"PAD_id is {tok.pad_id}")
print(f"BOS_id is {tok.bos_id}")
print(f"EOS_id is {tok.eos_id}")
print(f"Vocab size is {tok.vocab_size}")

/usr/local/lib/python3.10/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error reading MIDI file: running status without last_status
Error reading MIDI file: /content/midis/16907.mid
Error reading MIDI file: data byte must be in range 0..127
Error reading MIDI file: /content/midis/2651.mid
PAD_id is 285
BOS_id is 1
EOS_id is 0
Vocab size is 286


## Padding

In [ ]:
padded_tokens = tok.pad(encoded_tokens)
print(f"Maximum sequence len is {tok.seq_length}")

Maximum sequence len is 1412


---

# Training

In [ ]:
#@title Install `keras_nlp`
%%capture
!pip install keras_nlp

In [ ]:
import keras_nlp.layers as nlp_layers

def create_transformer(vocab_size, seq_len, embedding_dim, num_heads, dff, num_layers):
  # Input
    inputs = tf.keras.Input(shape=(seq_len,))

    # Embedding
    embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)

    # Encoder
    encoder = nlp_layers.TransformerEncoder(num_heads=num_heads, intermediate_dim=dff)(embedding)

    # Decoder
    decoder = nlp_layers.TransformerDecoder(num_heads=num_heads, intermediate_dim=dff)(embedding, encoder)

    # Output
    outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder)

    # Crea il modello
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model, encoder, decoder

In [ ]:
vocab_size = tok.vocab_size
seq_len = padded_tokens.shape[1]
import tensorflow as tf


model, encoder, decoder = create_transformer(vocab_size=vocab_size,
                                             seq_len=seq_len,
                                             embedding_dim=256,
                                             num_heads=8,
                                             dff=1024,
                                             num_layers=6)

In [ ]:
train_perc = 0.8
train_size = int(train_perc * padded_tokens.shape[0])

normalized_train_x = padded_tokens[:train_size]
normalized_val_x = padded_tokens[train_size:]

normalized_train_y = np.roll(normalized_train_x, shift=-1, axis=1)
normalized_val_y = np.roll(normalized_val_x, shift=-1, axis=1)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(normalized_train_x, normalized_train_y,
          epochs=100,
          validation_data=(normalized_val_x, normalized_val_y),
          callbacks=[early_stopping],
          batch_size=32
          )

model.save("NesGen_v1.keras")

Epoch 1/100
1474/1474 ━━━━━━━━━━━━━━━━━━━━ 1511s 1s/step - accuracy: 0.4578 - loss: 2.1846 - val_accuracy: 0.4906 - val_loss: 1.8453
Epoch 2/100
1263/1474 ━━━━━━━━━━━━━━━━━━━━ 3:09 897ms/step - accuracy: 0.4882 - loss: 1.8377

---

# Generation

In [ ]:
from tqdm import tqdm

def get_random_ids_from_dataset(dataset):
  return dataset[np.random.choice(dataset.shape[0])]

def sample_next_token(probabilities) -> int:
  # Ensure the probabilities sum to 1 (they should, but numerical issues might affect it)
  last_token_probs = probabilities[0, -1]
  last_token_probs /= last_token_probs.sum()
  return np.random.choice(len(last_token_probs), p=last_token_probs)

def next_token(model, seed_ids):
    probabilities = model.predict(seed_ids, verbose=0)
    next_token = sample_next_token(probabilities)
    return next_token

def generate_ids(model, seed_ids, eos_id, pad_id, bos_id, max_len=None, show_progress=True):
  if max_len is None:
    max_len = seed_ids.shape[1]
  seed = seed_ids
  generated_ids = []
  if not show_progress:
    iterations = range(max_len)
  else:
    iterations = tqdm(range(max_len))

  for _ in iterations:
    next_token_id = next_token(model, seed)
    generated_ids.append(next_token_id)
    if next_token_id == eos_id:
      break
    elif next_token_id == pad_id:
      continue

    seed = np.roll(seed, -1, axis=1)
    seed[0, -1] = next_token_id

  result = np.array(generated_ids)
  result[0] = bos_id
  result[-1] = eos_id
  return result

In [ ]:
seed = get_random_ids_from_dataset(normalized_train_x).reshape((1, seq_len))
generated_ids = generate_ids(
    model,
    seed,
    eos_id=tok.eos_id,
    pad_id=tok.pad_id,
    bos_id=tok.bos_id,
    max_len=500
)
print("\nGenerated\n" + str(generated_ids))

100%|██████████| 500/500 [00:40<00:00, 12.35it/s]


Generated
[  1 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285
 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285
 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285
 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285
 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285 285
   1   2 229   4 281 201   4   3   2 228   4 283 239   4 199   4 233  91
 227  53 271 230   6 281 235  41 199   4 260 229   4 226  82 194   4 194
   4 283 213   4 260 229   4 187 112 263 225  72 276 226   5 279 208   4
 281 216   4 260 239   4 199   4 280 213   4 259 211  99 281 187  72 281
 222  72 268 228   4 281 219   5 281 187 213   5 281 229  53 271 204   4
 283 220  47 260 229   4 281 194   4 144 259 219 106   3 229   4 282   4
   3 234   4 207  99 281 194   4 199   4 201   4 282   4 235  86 281 229
  72 259 201   4 260 219  72 281 241  88 260 217 125 261 236   4 281 213
   4 260 223   5   2 190  99 281 194 112

In [ ]:
def ids_to_midi(
    ids: np.ndarray,
    tokenizer: Tokenizer,
    file_name: str ="result.mid",
  ):
  tokenizer.decode([ids], file_name)

In [ ]:
ids_to_midi(generated_ids, tok)

['Start' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad'
 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad'
 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad'
 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad'
 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad'
 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad'
 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad'
 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Pad' 'Start' 'bar' 'note_70' 'len_0'
 'pos_6' 'note_42' 'len_0' 'beat' 'bar' 'note_69' 'len_0' 'pos_8'
 'note_80' 'len_0' 'note_40' 'len_0' 'note_74' 'len_44' 'note_68' 'len_23'
 'pos_24' 'note_71' 'len_10' 'pos_6' 'note_76' 'len_19' 'note_40' 'len_0'
 'pos_10' 'note_70' 'len_0' 'note_67' 'len_38' 'note_35' 'len_0' 'note_35'
 'len_0' 'pos_8' 'note_54' 'len_0' 'pos_10' 'note_70' 'len_0' 'note_28'
 'len_6' 'pos_13' 'note_66' 'len_3' 'pos_3' 'note_67' 'l